In [8]:
from bs4 import BeautifulSoup
import concurrent.futures
import pandas as pd
import requests
import os

ROOT_URL="https://en.wikipedia.org" 
URL=f"{ROOT_URL}/wiki/List_of_Academy_Award%E2%80%93winning_films"
NOMINEES_PATH = os.path.join(os.getcwd(), "oscar_nominees")

In [9]:
nominees_response=requests.get(URL)
nominees_html=nominees_response.content.decode()
nominees_soup=BeautifulSoup(nominees_html, 'html.parser')
nominees_table=nominees_soup.select_one(".wikitable")
nominees_links=[a["href"] for a in nominees_table.select("a") if "_in_film" not in a["href"] and "#" not in a["href"]]

In [ ]:
# EXECUTE ONLY IF IT FOLDER DOES NOT EXISTS

get_nominee_page=lambda link: requests.get(ROOT_URL + link).content.decode()
write_page_to_file=lambda link, index: open(os.path.join(NOMINEES_PATH, f"{index}.html"), "w").write(get_nominee_page(link))

def get_nominees_pages(link: str, index: int):
    page = requests.get(ROOT_URL + link)
    with open(os.path.join(NOMINEES_PATH, f"{index}.html"), "w") as f:
        f.write(page.content.decode())

if os.path.isdir(NOMINEES_PATH):
    os.removedirs(NOMINEES_PATH)

os.makedirs(NOMINEES_PATH)

with concurrent.futures.ThreadPoolExecutor(max_workers=5) as executor:
    futures = [executor.submit(write_page_to_file, link, index) for index, link in enumerate(nominees_links)]
    for future in futures:
        future.result()

In [70]:
nominee_pages = []
def get_nominees_pages_as_bs4(doc: str):
    if ".html" in doc:
        nominee_pages.append(BeautifulSoup(open(os.path.join(NOMINEES_PATH, doc), 'r').read(),"html.parser"))

with concurrent.futures.ThreadPoolExecutor(max_workers=5) as executor:
    futures = [executor.submit(get_nominees_pages_as_bs4, file) for file in os.listdir(NOMINEES_PATH)]
    for future in futures:
        future.result()

In [ ]:
nominee_pages